In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/twitter-suicidal-intention-dataset/master/twitter-suicidal_data.csv')
df.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [ ]:
df.shape

(9119, 2)

In [ ]:
df['intention'].value_counts()

0    5121
1    3998
Name: intention, dtype: int64

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-wkg0se7s
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-wkg0se7s
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7639 sha256=56fa1f6bf0405612b0bcca93ecee8242e8797b90c7d3b760badfd6d51a03011b
  Stored in directory: /tmp/pip-ephem-wheel-cache-cyj6hwef/wheels/d1/c3/bb/559fe93e652b51cbc532f17e9693f3b70055f8560cf06c1fb3
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: get_clean(x))

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [ ]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,3), analyzer='char')

In [ ]:
X = tfidf.fit_transform(df['tweet'])
y = df['intention']

In [ ]:
X.shape

(9119, 10404)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.classes_

array([0, 1])

In [ ]:
y_pred

array([0, 1, 0, ..., 1, 1, 1])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1060
           1       0.91      0.91      0.91       764

    accuracy                           0.92      1824
   macro avg       0.92      0.92      0.92      1824
weighted avg       0.92      0.92      0.92      1824



In [ ]:
x = 'no one cares about me. i will die alone'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([1])

In [ ]:
x = 'Today I am so happy, Thanks for making my day good'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([0])

In [ ]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import joblib from the joblib module
import joblib

# Save the model object using joblib
joblib.dump(clf, 'model.joblib')

# Load the model object using joblib
clf = joblib.load('model.joblib')


In [ ]:
x = 'Today I am so sad, want to kill myself'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([1])